# Імпортування даних у таблиці вимірів сховища даних

Імпортуємо бібліотеки

In [99]:
import pandas as pd
from dotenv import dotenv_values
from sqlalchemy import create_engine, text
from sqlalchemy.sql import select

Завантажуємо змінні оточення з .env файлу для з'єднання зі сховищем

In [64]:
config = dotenv_values()

DB_NAME = config.get('DB_NAME')
DB_USER = config.get('DB_USER')
DB_HOST = config.get('DB_HOST')
DB_PASSWORD = config.get('DB_PASSWORD')
DB_PORT = config.get('DB_PORT')

З'єднуємося з базою даних

In [75]:
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

## Countries Table

Імпортуємо дані з датасету

In [70]:
countries_df = pd.read_csv('../data/f1db-countries.csv')
countries_df.head()

,id,alpha2Code,alpha3Code,name,demonym,continentId
0,afghanistan,AF,AFG,Afghanistan,Afghan,asia
1,aland-islands,AX,ALA,Åland Islands,Ålandic,europe
2,albania,AL,ALB,Albania,Albanian,europe
3,algeria,DZ,DZA,Algeria,Algerian,africa
4,american-samoa,AS,ASM,American Samoa,Samoan,australia


Приводимо дані у потрібний нам формат для сховища

In [71]:
countries_df = countries_df[['alpha3Code', 'name', 'demonym', 'id']]
countries_df.head()

,alpha3Code,name,demonym,id
0,AFG,Afghanistan,Afghan,afghanistan
1,ALA,Åland Islands,Ålandic,aland-islands
2,ALB,Albania,Albanian,albania
3,DZA,Algeria,Algerian,algeria
4,ASM,American Samoa,Samoan,american-samoa


Перейменуємо стовпці

In [74]:
countries_df.columns = ['alpha3_code', 'name', 'demonym', 'id']
countries_df.head()

,alpha3_code,name,demonym,id
0,AFG,Afghanistan,Afghan,afghanistan
1,ALA,Åland Islands,Ålandic,aland-islands
2,ALB,Albania,Albanian,albania
3,DZA,Algeria,Algerian,algeria
4,ASM,American Samoa,Samoan,american-samoa


Створимо функцію, яка внесе потрібні нам дані у сховище та поверне згенеровані ідентифікатори для кожного рядка

In [149]:
def insert_data(df, table):
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT COUNT(*) FROM {table}"))
        count = result.scalar()

        if count == 0:
            df.to_sql(table, engine, if_exists='append', index=False)

        result = connection.execute(text(f"SELECT id FROM {table}"))
        generated_ids = [row[0] for row in result]
        
    return generated_ids

Вносимо дані з датафрейму у таблицю країн, функція повертає датафрейм з ідентифікаторами, які сховище присвоїло рядкам. Ці ідентифікатори додаємо до датафрейму для того, аби в наступних датафреймах замінити їх.

In [129]:
countries_df['dbId'] = insert_data(countries_df[['alpha3_code', 'name', 'demonym']], 'countries')
countries_df.head()

,alpha3_code,name,demonym,id,dbId
0,AFG,Afghanistan,Afghan,afghanistan,1
1,ALA,Åland Islands,Ålandic,aland-islands,2
2,ALB,Albania,Albanian,albania,3
3,DZA,Algeria,Algerian,algeria,4
4,ASM,American Samoa,Samoan,american-samoa,5


## Circuits Table

Імпортуємо дані з датасету

In [141]:
circuits_df = pd.read_csv('../data/f1db-circuits.csv')
circuits_df.head()

,id,name,fullName,previousNames,type,placeName,countryId,latitude,longitude,totalRacesHeld
0,adelaide,Adelaide,Adelaide Street Circuit,NaN,STREET,Adelaide,australia,-34.927222,138.617222,11
1,aida,Aida,Okayama International Circuit,TI Circuit Aida,RACE,Aida,japan,34.915000,134.221111,2
2,ain-diab,Ain-Diab,Ain-Diab Circuit,NaN,ROAD,Casablanca,morocco,33.578611,-7.687500,1
3,aintree,Aintree,Aintree Motor Racing Circuit,NaN,ROAD,Aintree,united-kingdom,53.476944,-2.940556,5
4,anderstorp,Anderstorp Raceway,Anderstorp Raceway,Scandinavian Raceway,RACE,Anderstorp,sweden,57.264167,13.601389,6


Замінимо ідентифікатори країн у таблиці на ті, що згенерувало сховище.

In [142]:
circuits_df['countryId'] = circuits_df.merge(countries_df[['id', 'dbId']], left_on='countryId', right_on='id', how='left')['dbId']
circuits_df.head()

,id,name,fullName,previousNames,type,placeName,countryId,latitude,longitude,totalRacesHeld
0,adelaide,Adelaide,Adelaide Street Circuit,NaN,STREET,Adelaide,14,-34.927222,138.617222,11
1,aida,Aida,Okayama International Circuit,TI Circuit Aida,RACE,Aida,113,34.915000,134.221111,2
2,ain-diab,Ain-Diab,Ain-Diab Circuit,NaN,ROAD,Casablanca,149,33.578611,-7.687500,1
3,aintree,Aintree,Aintree Motor Racing Circuit,NaN,ROAD,Aintree,235,53.476944,-2.940556,5
4,anderstorp,Anderstorp Raceway,Anderstorp Raceway,Scandinavian Raceway,RACE,Anderstorp,216,57.264167,13.601389,6


Приводимо дані у потрібний нам формат для сховища

In [143]:
circuits_df = circuits_df[['name', 'fullName', 'type', 'countryId', 'latitude', 'longitude', 'id']]
circuits_df.head()

,name,fullName,type,countryId,latitude,longitude,id
0,Adelaide,Adelaide Street Circuit,STREET,14,-34.927222,138.617222,adelaide
1,Aida,Okayama International Circuit,RACE,113,34.915000,134.221111,aida
2,Ain-Diab,Ain-Diab Circuit,ROAD,149,33.578611,-7.687500,ain-diab
3,Aintree,Aintree Motor Racing Circuit,ROAD,235,53.476944,-2.940556,aintree
4,Anderstorp Raceway,Anderstorp Raceway,RACE,216,57.264167,13.601389,anderstorp


Перейменуємо стовпці

In [144]:
circuits_df.columns = ['name', 'full_name', 'type', 'country_id', 'latitude', 'longitude', 'id']
circuits_df.head()

,name,full_name,type,country_id,latitude,longitude,id
0,Adelaide,Adelaide Street Circuit,STREET,14,-34.927222,138.617222,adelaide
1,Aida,Okayama International Circuit,RACE,113,34.915000,134.221111,aida
2,Ain-Diab,Ain-Diab Circuit,ROAD,149,33.578611,-7.687500,ain-diab
3,Aintree,Aintree Motor Racing Circuit,ROAD,235,53.476944,-2.940556,aintree
4,Anderstorp Raceway,Anderstorp Raceway,RACE,216,57.264167,13.601389,anderstorp


Вносимо дані з датафрейму у таблицю гоночних трас

In [150]:
circuits_df['dbId'] = insert_data(circuits_df[['name', 'full_name', 'type', 'country_id', 'latitude', 'longitude']], 'circuits')
circuits_df.head()

,name,full_name,type,country_id,latitude,longitude,id,dbId
0,Adelaide,Adelaide Street Circuit,STREET,14,-34.927222,138.617222,adelaide,1
1,Aida,Okayama International Circuit,RACE,113,34.915000,134.221111,aida,2
2,Ain-Diab,Ain-Diab Circuit,ROAD,149,33.578611,-7.687500,ain-diab,3
3,Aintree,Aintree Motor Racing Circuit,ROAD,235,53.476944,-2.940556,aintree,4
4,Anderstorp Raceway,Anderstorp Raceway,RACE,216,57.264167,13.601389,anderstorp,5


## Constructors Table

Імпортуємо дані з датасету

In [151]:
constructors_df = pd.read_csv('../data/f1db-constructors.csv')
constructors_df.head()

,id,name,fullName,countryId,bestChampionshipPosition,bestStartingGridPosition,bestRaceResult,totalChampionshipWins,totalRaceEntries,totalRaceStarts,totalRaceWins,total1And2Finishes,totalRaceLaps,totalPodiums,totalPodiumRaces,totalChampionshipPoints,totalPolePositions,totalFastestLaps
0,adams,Adams,Adams,united-states-of-america,NaN,17.0,27.0,0,1,1,0,0,108,0,0,0.0,0,0
1,afm,AFM,Alex von Falkenhausen Motorenbau,germany,NaN,9.0,9.0,0,4,4,0,0,98,0,0,0.0,0,0
2,ags,AGS,Automobiles Gonfaronnaises Sportives,france,11.0,10.0,6.0,0,80,48,0,0,2031,0,0,2.0,0,0
3,alfa-romeo,Alfa Romeo,Alfa Romeo Racing,italy,6.0,1.0,1.0,0,214,214,10,4,20608,26,18,199.0,12,16
4,alfa-special,Alfa Special,Alfa Special,south-africa,NaN,16.0,10.0,0,2,2,0,0,132,0,0,0.0,0,0


Замінимо ідентифікатори країн у таблиці на ті, що згенерувало сховище.

In [152]:
constructors_df['countryId'] = constructors_df.merge(countries_df[['id', 'dbId']], left_on='countryId', right_on='id', how='left')['dbId']
constructors_df.head()

,id,name,fullName,countryId,bestChampionshipPosition,bestStartingGridPosition,bestRaceResult,totalChampionshipWins,totalRaceEntries,totalRaceStarts,totalRaceWins,total1And2Finishes,totalRaceLaps,totalPodiums,totalPodiumRaces,totalChampionshipPoints,totalPolePositions,totalFastestLaps
0,adams,Adams,Adams,237,NaN,17.0,27.0,0,1,1,0,0,108,0,0,0.0,0,0
1,afm,AFM,Alex von Falkenhausen Motorenbau,84,NaN,9.0,9.0,0,4,4,0,0,98,0,0,0.0,0,0
2,ags,AGS,Automobiles Gonfaronnaises Sportives,77,11.0,10.0,6.0,0,80,48,0,0,2031,0,0,2.0,0,0
3,alfa-romeo,Alfa Romeo,Alfa Romeo Racing,110,6.0,1.0,1.0,0,214,214,10,4,20608,26,18,199.0,12,16
4,alfa-special,Alfa Special,Alfa Special,206,NaN,16.0,10.0,0,2,2,0,0,132,0,0,0.0,0,0


Приводимо дані у потрібний нам формат для сховища

In [156]:
constructors_df = constructors_df[['name', 'fullName', 'countryId', 'id']]
constructors_df.head()

,name,fullName,countryId,id
0,Adams,Adams,237,adams
1,AFM,Alex von Falkenhausen Motorenbau,84,afm
2,AGS,Automobiles Gonfaronnaises Sportives,77,ags
3,Alfa Romeo,Alfa Romeo Racing,110,alfa-romeo
4,Alfa Special,Alfa Special,206,alfa-special


Перейменуємо стовпці

In [159]:
constructors_df.columns = ['name', 'full_name', 'country_id', 'id']
constructors_df.head()

,name,full_name,country_id,id
0,Adams,Adams,237,adams
1,AFM,Alex von Falkenhausen Motorenbau,84,afm
2,AGS,Automobiles Gonfaronnaises Sportives,77,ags
3,Alfa Romeo,Alfa Romeo Racing,110,alfa-romeo
4,Alfa Special,Alfa Special,206,alfa-special


Вносимо дані з датафрейму у таблицю команд

In [160]:
constructors_df['dbId'] = insert_data(constructors_df[['name', 'full_name', 'country_id']], 'constructors')
constructors_df.head()

,name,full_name,country_id,id,dbId
0,Adams,Adams,237,adams,1
1,AFM,Alex von Falkenhausen Motorenbau,84,afm,2
2,AGS,Automobiles Gonfaronnaises Sportives,77,ags,3
3,Alfa Romeo,Alfa Romeo Racing,110,alfa-romeo,4
4,Alfa Special,Alfa Special,206,alfa-special,5


## Drivers Table

## Grand Prix Table

## Races Table

## Seasons Table